In [1]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from BiasedErasure.delayed_erasure_decoders.Experimental_Loss_Decoder import *
from BiasedErasure.main_code.Simulator import *
from BiasedErasure.main_code.noise_channels import atom_array
from BiasedErasure.delayed_erasure_decoders.HeraldedCircuit_SWAP_LD import HeraldedCircuit_SWAP_LD


In [9]:
def simulate(noise_params={}, chunk_size=10000, noise_scales=[1.,1.,1.,1.], distance=5):
    cycles = 4
    decoder_basis = 'Z'
    num_rounds = 5
    gate_ordering = ['N', 'Z', 'Zr', 'Nr']
    Meta_params = {'architecture': 'CBQC', 'code': 'Rotated_Surface', 'logical_basis': decoder_basis, 'bias_preserving_gates': 'False', 
        'noise': 'atom_array', 'is_erasure_biased': 'False', 'LD_freq': '100', 'LD_method': 'None', 'SSR': 'True', 'cycles': str(num_rounds - 1),
            'ordering': gate_ordering,
            'decoder': 'ML',
        'circuit_type': 'memory', 'Steane_type': 'regular', 'printing': 'False', 'num_logicals': '1', 'loss_decoder': 'independent', 'obs_pos': 'd-1', 'n_r': '0'}
    bloch_point_params = {'erasure_ratio': '1', 'bias_ratio': '0.5'}
    
    print(noise_params)
    # Example data, change this:
    dx = distance
    dy = distance
    
    output_dir = ""
    simulator = Simulator(Meta_params=Meta_params, atom_array_sim=True, 
                                bloch_point_params=bloch_point_params, noise=atom_array , 
                                phys_err_vec=None, loss_detection_method=HeraldedCircuit_SWAP_LD, 
                                cycles = cycles, output_dir=output_dir, save_filename=None, save_data_during_sim=False)
    
    meas, det = simulator.sampling_with_loss(chunk_size, dx, dy, None, noise_params=noise_params)
    
    # meas, det = simulator.sampling_with_loss_old(chunk_size, dx, dy, None, noise_params=noise_params)
    return torch.Tensor(meas).int(), torch.Tensor(det).int()

In [10]:
def p_ij_matrix(detections: np.ndarray, d=5) -> np.ndarray:
    num_ancillas = d**2 - 1
    num_time_steps = 4
    total_steps = num_ancillas * num_time_steps
    
    # Initialize the correlation matrix
    p_ij = np.zeros((total_steps, total_steps))

    # Precompute the means for all columns
    x_avg = np.mean(detections, axis=0)
    
    # Precompute the pairwise products
    # xixj_avg_matrix = np.dot(detections.T, detections) / detections.shape[0]
    xixj_avg_matrix = np.mean(np.einsum('ij,ik->ijk', detections, detections), axis=0)
    # Calculate the correlation matrix
    for idx_i in range(total_steps):
        for idx_j in range(idx_i + 1, total_steps):
            xi_avg = x_avg[idx_i]
            xj_avg = x_avg[idx_j]
            xixj_avg = xixj_avg_matrix[idx_i, idx_j]
            
            numerator = 4 * (xixj_avg - xi_avg * xj_avg)
            denominator = 1 - 2 * xi_avg - 2 * xj_avg + 4 * xixj_avg
            
            # Avoid dividing by zero or taking sqrt of a negative number
            if denominator > 0 and 1 - numerator / denominator >= 0:
                value = 0.5 - 0.5 * np.sqrt(1 - numerator / denominator)
                p_ij[idx_i, idx_j] = value
                p_ij[idx_j, idx_i] = value  # The matrix is symmetric

    return np.round(p_ij, 3)


detector_order = np.array([
    *[0,1,3,5,8,10,13,15,18,20,22,23],
    *list(range(24)), *list(range(24)), *list(range(24)),
    *[0,1,3,5,8,10,13,15,18,20,22,23]
])
mapper = sum([list(np.argwhere(detector_order == detector_idx).flatten()) for detector_idx in range(24)], start=[])




In [11]:
# from tqdm.auto import *
# maximums = np.array([
#         3e-6, *[8e-6]*3, *[0.05/4]*3, *[0.05/4]*15, 0.05/4, *[1e-3]*3, 0.03, 0.04 
#     ])
# to_dict = lambda noise_params: dict(
#         idle_loss_rate=noise_params[0],
#         idle_error_rate=noise_params[1:4],
#         entangling_zone_error_rate=noise_params[4:7],
#         entangling_gate_error_rate=noise_params[7:22],
#         entangling_gate_loss_rate=noise_params[22],
#         single_qubit_error_rate=noise_params[23:26],
#         reset_error_rate=noise_params[26],
#         measurement_error_rate=noise_params[27]
#     )
# keys = ['idle_loss_rate', *['idle_error_rate']*3, *['entangling_zone_error_rate']*3, 
#         *['entangling_gate_error_rate']*15, 'entangling_gate_loss_rate', *['single_qubit_error_rate']*3,
#         'reset_error_rate', 'measurement_error_rate'
#         ]
# distance = 3
# fig, axes = plt.subplots(4,7,figsize=(10,5))
# axes = axes.flatten()
# for i in trange(28):
#     x = np.zeros(28)
#     x[i] = maximums[i]/5
#     meas, det = simulate(noise_params=to_dict(x), chunk_size=20000)
#     cov_ = p_ij_matrix(det.numpy(), d=distance).flatten()
#     cov_ /= np.linalg.norm(cov_)
#     tmp = int(np.sqrt(len(cov_)))
#     axes[i].imshow(cov_.reshape((tmp,tmp)))
#     axes[i].axis('off')
#     axes[i].set_title(f'{keys[i]}', fontsize=5)

# fig.tight_layout()

In [12]:
from tqdm.auto import *
maximums = np.array([
        3e-6, *[8e-6]*3, *[0.05/4]*3, *[0.05/4]*15, 0.05/4, *[1e-3]*3, 0.03, 0.04 
    ])
to_dict = lambda noise_params: dict(
        idle_loss_rate=noise_params[0],
        idle_error_rate=noise_params[1:4],
        entangling_zone_error_rate=noise_params[4:7],
        entangling_gate_error_rate=noise_params[7:22],
        entangling_gate_loss_rate=noise_params[22],
        single_qubit_error_rate=noise_params[23:26],
        reset_error_rate=noise_params[26],
        measurement_error_rate=noise_params[27]
    )
keys = ['idle_loss_rate', *['idle_error_rate']*3, *['entangling_zone_error_rate']*3, 
        *['entangling_gate_error_rate']*15, 'entangling_gate_loss_rate', *['single_qubit_error_rate']*3,
        'reset_error_rate', 'measurement_error_rate'
        ]
fig  = plt.figure()
d = 5
distance = d
decoder_basis = 'Z'
num_rounds = 5


data_source = 't'


if data_source == 'e':
    # import from the experiment:
    output_dir = "/Users/gefenbaranes/Lukin SiV Dropbox/Gefen Baranes/RepetitiveQEC/Analysis/Data/July2024_repetitiveQEC_preprocessing/d%i/r%i/%s/final_data"%(distance,num_rounds - 1,decoder_basis)
    det_exp = np.load(f"{output_dir}/experimental_detection_events{decoder_basis}.npy")
    print(det_exp.shape)
    cov = p_ij_matrix(det_exp)
    cov /= np.linalg.norm(cov)
    
# theory:
if data_source == 't':
    i = 3 # specific error only
    x = np.zeros(28)
    x[i] = maximums[i]/5
    x = maximums / 5
    meas, det = simulate(noise_params=to_dict(x), chunk_size=3000, distance = d)

    print(det.numpy().shape)

    detection_events = det_exp if data_source == 'e' else det

    
    cov = p_ij_matrix(detection_events.numpy(), d)
    cov /= np.linalg.norm(cov)



ii, jj = np.meshgrid(mapper, mapper)
num_detector_appearances = [len(np.argwhere(detector_order == detector_idx).flatten()) for detector_idx in range(24)]

plt.figure(figsize=(20,20))
plt.matshow(cov[ii,jj])
positions = np.cumsum(num_detector_appearances)
middle_position = list((np.ceil((positions + np.concatenate([[0], positions[:-1]]))/2)).astype(int))
ticks = np.arange(96)
labels = ['' if tick not in middle_position else f'D{middle_position.index(tick)}' for tick in ticks]
plt.xticks(ticks, labels, rotation=90)
plt.yticks(ticks, labels)
for position in positions:
    plt.axhline(position-0.5, c='white', lw=1, alpha=0.3)
    plt.axvline(position-0.5, c='white', lw=1, alpha=0.3)
plt.savefig('./cov.pdf', bbox_inches='tight')



fig.tight_layout()

{'idle_loss_rate': 6e-07, 'idle_error_rate': array([1.6e-06, 1.6e-06, 1.6e-06]), 'entangling_zone_error_rate': array([0.0025, 0.0025, 0.0025]), 'entangling_gate_error_rate': array([0.0025, 0.0025, 0.0025, 0.0025, 0.0025, 0.0025, 0.0025, 0.0025,
       0.0025, 0.0025, 0.0025, 0.0025, 0.0025, 0.0025, 0.0025]), 'entangling_gate_loss_rate': 0.0025, 'single_qubit_error_rate': array([0.0002, 0.0002, 0.0002]), 'reset_error_rate': 0.006, 'measurement_error_rate': 0.008}
entangling Pauli error rate = None, entangling loss rate = None
Using orderings: ['N', 'Z', 'Zr', 'Nr']
0.0
0.0
/loss_circuits/CBQC__Rotated_Surface__memory__1log__Z__0__atom_array__0__LD_freq_100__SSR_1__LD_method_None__ordering_N_Z_Zr_Nr/dx_5__dy_5__c_4
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95

In [46]:
detection_events.

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)